Sample:

Link: https://eparlib.nic.in/handle/123456789/6/browse?page-token=2d1d323f8c3e&page-token-value=736c7528ed63b9874a20690824001b9d&type=loksabhanumber&submit_browse=Lok+Sabha+Number

Main page of all lok sabhas (1-16) ---> speeches list ----> get their links -----> open the links ----> download the pdfs 

This notebook has the code to collect all the links of the the web pages of the sessions. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import math
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
website = 'https://eparlib.nic.in/handle/123456789/6/browse?page-token=ccdcdf0f8ccb&page-token-value=5f621e68b55db83daa4ebdcd11624225&type=loksabhanumber&submit_browse=Lok+Sabha+Number'

page = requests.get(website, stream=True, verify=False)
soup = BeautifulSoup(page.text, 'html.parser')

In [4]:
sessions_link = soup.findAll(class_='list-group-item')

In [5]:
links = []
values = []
for link in sessions_link:
    links.append('https://eparlib.nic.in' + link.find('a')['href'])    
    values.append(link.find('span', class_='badge').contents[0])

In [6]:
links = links[11:]
links

['https://eparlib.nic.in/handle/123456789/6/browse?type=loksabhanumber&order=ASC&rpp=20&value=12',
 'https://eparlib.nic.in/handle/123456789/6/browse?type=loksabhanumber&order=ASC&rpp=20&value=13',
 'https://eparlib.nic.in/handle/123456789/6/browse?type=loksabhanumber&order=ASC&rpp=20&value=14',
 'https://eparlib.nic.in/handle/123456789/6/browse?type=loksabhanumber&order=ASC&rpp=20&value=15',
 'https://eparlib.nic.in/handle/123456789/6/browse?type=loksabhanumber&order=ASC&rpp=20&value=16']

In [7]:
values = values[11:]
values

['88', '523', '331', '357', '331']

In [8]:
s = 0
for value in values:
    value = int(value)
    s += value
print(s)

1630


In [36]:
def get_transcripts_links_session(link, value):
    total_pages = math.ceil(int(value)/20)
    count = 0
    all_index_pages = [link]
    all_pages = []
    k = 1
    while count<total_pages:
        try:
            page = requests.get(link, stream=True, verify=False)
            soup = BeautifulSoup(page.text, 'html.parser')
            next_page_link = soup.find('a', class_='pull-right')['href']
            #print(next_page_link)
            link = 'https://eparlib.nic.in'+ next_page_link
            all_index_pages.append(link)
        except TypeError:
            k = 1
        count += 1
    count = 0
    for link in all_index_pages:
        page = requests.get(link, stream=True, verify=False)
        soup = BeautifulSoup(page.text, 'html.parser')
        table = soup.find('table', {'class':'panel'})
        table_links = table.findAll('a')
        for web_link in table_links:
            count += 1
            #print(count, web_link.get('href'))
            all_pages.append(web_link.get('href'))
    return all_pages

In [37]:
url = 'https://eparlib.nic.in/handle/123456789/6/browse?type=loksabhanumber&order=ASC&rpp=20&value=12'
page = requests.get(link, stream=True, verify=False)
soup = BeautifulSoup(page.text, 'html.parser')
table = soup.find('table', {'class':'panel'})
table.findAll('a')[0].get('href')

'/handle/123456789/758034?view_type=browse'

In [38]:
all_pages = []
count = 0
for link in links:
    temp_pages = get_transcripts_links_session(link, values[count])
    all_pages += temp_pages
    count += 1 
    print(count, end=" ")

1 2 3 4 5 

In [39]:
df = pd.DataFrame(all_pages)

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1630 entries, 0 to 1629
Data columns (total 1 columns):
0    1630 non-null object
dtypes: object(1)
memory usage: 12.8+ KB


In [41]:
df.to_csv('pages.csv')